<a href="https://colab.research.google.com/github/sushaanpatel/work-test/blob/main/agent_test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -q -U bitsandbytes
!pip install -q -U transformers
!pip install -q -U accelerate

In [ ]:
# !pip install transformers==4.46.2 peft==0.13.2 accelerate==1.1.1 trl==0.12.1 bitsandbytes==0.45.2 datasets==3.1.0 huggingface-hub==0.27.0 safetensors==0.4.5 -q

In [ ]:
# !pip install -q -U bitsandbytes==0.45.3
# !pip install -q -U git+https://github.com/huggingface/transformers.git
# !pip install -q -U git+https://github.com/huggingface/peft.git
# !pip install -q -U git+https://github.com/huggingface/accelerate.git

In [ ]:
import torch
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    AutoTokenizer
    # pipeline
)

In [ ]:
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    A token is already saved on your machine. Run `huggingface-cli whoami` to get more information or `huggingface-cli logout` if you want to log out.
    Setting a new token will erase the existing one.
    To log in, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): 
Add token as git credential? (Y/n) y
Traceback (most recent call last)

In [ ]:
model_name = "mistralai/Mistral-7B-Instruct-v0.1"
#Tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=True)
tokenizer.pad_token = tokenizer.unk_token
tokenizer.pad_token_id =  tokenizer.unk_token_id
tokenizer.padding_side = 'left'

tokenizer_config.json:   0%|          | 0.00/2.10k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

In [ ]:
compute_dtype = getattr(torch, "float16")
bnb_config = BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_quant_type="nf4",
        bnb_4bit_compute_dtype=compute_dtype,
        bnb_4bit_use_double_quant=True,
)
model = AutoModelForCausalLM.from_pretrained(
          model_name, quantization_config=bnb_config, device_map={"": 0}
)
#Configure the pad token in the model
model.config.pad_token_id = tokenizer.pad_token_id



config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

CUDA is required but not available for bitsandbytes. Please consider installing the multi-platform enabled version of bitsandbytes, which is currently a work in progress. Please check currently supported platforms and installation instructions at https://huggingface.co/docs/bitsandbytes/main/en/installation#multi-backend


RuntimeError: CUDA is required but not available for bitsandbytes. Please consider installing the multi-platform enabled version of bitsandbytes, which is currently a work in progress. Please check currently supported platforms and installation instructions at https://huggingface.co/docs/bitsandbytes/main/en/installation#multi-backend

In [ ]:
!pip install langchain_community

In [ ]:
from langchain.llms import HuggingFacePipeline
from transformers import pipeline

text_generation_pipeline = pipeline(
    model=model,
    tokenizer=tokenizer,
    task="text-generation",
    temperature=0.2,
    repetition_penalty=1.1,
    return_full_text=True,
    max_new_tokens=1000,
)

mistral_llm = HuggingFacePipeline(pipeline=text_generation_pipeline)

In [ ]:
!pip install playwright

In [ ]:
!python -m playwright install
!python -m playwright install-deps

In [ ]:
!pip install html2text
!pip install faiss-gpu-cu12
!pip install pypdf

In [ ]:
from datasets import load_dataset
from peft import LoraConfig, PeftModel

from langchain.text_splitter import CharacterTextSplitter
from langchain.document_transformers import Html2TextTransformer
from langchain.document_loaders import AsyncChromiumLoader, PyPDFLoader

from langchain.embeddings.huggingface import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS

from langchain.prompts import PromptTemplate
from langchain.schema.runnable import RunnablePassthrough
from langchain.chains import LLMChain

In [ ]:
import nest_asyncio
nest_asyncio.apply()

# Articles to index
articles = ["https://www.fantasypros.com/2023/11/rival-fantasy-nfl-week-10/",
            "https://www.fantasypros.com/2023/11/5-stats-to-know-before-setting-your-fantasy-lineup-week-10/",
            "https://www.fantasypros.com/2023/11/nfl-week-10-sleeper-picks-player-predictions-2023/",
            "https://www.fantasypros.com/2023/11/nfl-dfs-week-10-stacking-advice-picks-2023-fantasy-football/",
            "https://www.fantasypros.com/2023/11/players-to-buy-low-sell-high-trade-advice-2023-fantasy-football/"]

# Scrapes the blogs above
loader = AsyncChromiumLoader(articles)
docs = loader.load()

In [ ]:
# html2text = Html2TextTransformer()
# docs_transformed = html2text.transform_documents(docs)

async def load_pages(loader):
  pages = []
  async for page in loader.alazy_load():
      pages.append(page)

  return pages

documents = ["/content/1aa42dac_form_submission_1747154194.pdf", "/content/SEAL%20Methodology_v1.0 (1).pdf"]
final = []
text_splitter = CharacterTextSplitter(chunk_size=100,
                                      chunk_overlap=0)

for document in documents:
  loader = PyPDFLoader(document)
  current_pdf = await load_pages(loader)
  chunked_documents = text_splitter.split_documents(current_pdf)
  final.extend(chunked_documents)


# print(type(docs[0]))
# print(type(docs_transformed[0]))

# Chunk text


print(type(chunked_documents[0]))

# Load chunked documents into the FAISS index
db = FAISS.from_documents(final,
                          HuggingFaceEmbeddings(model_name='sentence-transformers/all-mpnet-base-v2'))

retriever = db.as_retriever()

In [ ]:
#provide more context on what each document is, and provide context on the correct latex format
prompt_template = """
### [INST] Instruction: Using this context to inform all further answers:

{context}

### Instruction: Use knowledge from the Policy Section of the report to answer the following questions, please provide answer as a latex file with the appopriate format.
{question} [/INST]
 """

# Create prompt from prompt template
prompt = PromptTemplate(
    input_variables=["context", "question"],
    template=prompt_template,
)

# Create llm chain
llm_chain = LLMChain(llm=mistral_llm, prompt=prompt)

In [ ]:
rag_chain = (
 {"context": retriever, "question": RunnablePassthrough()}
    | llm_chain
)

result = rag_chain.invoke("What controls does the firm have to prevent unauthorized access to file rooms")

In [ ]:
result

In [ ]:
# print(result['text'].split('\n'))

for line in result['text'].split('\n'):
  print(line)

# flag = False
# for line in result['text'].split('\n'):
#   if flag:
#     print(line)

#   if '<answer>' in line:
#     flag = True

#   if '</answer>' in line:
#     flag = False

In [ ]:
!pip install pylatexenc

In [ ]:
text = r"""
    According to the Policy section of the report, the firm has the following controls to prevent unauthorized access to file rooms:
    \begin{itemize}
      \item Access to select personnel only
      \item Visitor log
      \item Monitoring (video, motion sensors etc.)
      \item Physical controls (locks, keypad access etc.)
    \end{itemize}"""

from pylatexenc.latex2text import LatexNodes2Text
latex = r"""\textbf{Hi there!} Here is \emph{an equation}:
\begin{equation}
    \zeta = x + i y
\end{equation}
where $i$ is the imaginary unit.
"""
print(LatexNodes2Text().latex_to_text(text))